### Preparar ambiente

Primero descomentar las siguientes lineas e instalar PyAthena. Solo se hace la primera vez que se abre la instancia

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

#### Lista de tablas de interes 

In [4]:
#tabla_datos_input = f"utc_world" # Tabla original https://timezonedb.com/download
tabla_datos_input = f"mobility" # Tabla original
tabla_datos_output_view = f"utc_latam_view" # Tabla modificada para continuar con la query
tabla_datos_output_table = f"utc_latam" # Tabla modificada para continuar con la query

#### Eliminando tablas  

In [5]:
for tabla in [ tabla_datos_output_view, tabla_datos_output_table]:
        query = (f"DROP TABLE IF EXISTS {tabla} ")
        cursor.execute(query)
        print(f" Se ha eliminado la vista: {tabla} ")

 Se ha eliminado la vista: utc_latam_view 
 Se ha eliminado la vista: utc_latam 


#### Generando tabla UTC Latam - Original

In [6]:
query = (f'''CREATE VIEW {tabla_datos_output_view}
AS SELECT col0 AS country_name,
        col1 AS country_code,
        col2 AS utc_original,
        col3 AS utc_hora,
        col4 AS utc_segundos
FROM {tabla_datos_input}
''')
cursor.execute(query)
print(f" Se ha creado la vista: {tabla_datos_output_view} ")

 Se ha creado la vista: utc_latam_view 


In [7]:
query = f'''SELECT country_code,
        COUNT(*) as n_obs
FROM {tabla_datos_output_view}
GROUP BY country_code 
ORDER BY country_code '''
cursor.execute(query).as_pandas()

,country_code,n_obs
0,AE,1
1,AF,1
2,AL,1
3,AM,1
4,AR,1
...,...,...
135,VE,1
136,VN,1
137,YE,1
138,ZA,1


#### Generando UTC

#### Generando tabla UTC Latam - Modificada

In [8]:
query = (f'''CREATE TABLE {tabla_datos_output_table}
WITH ( 
    external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-public/utc_latam', 
    format = 'PARQUET', 
    parquet_compression = 'SNAPPY') AS
SELECT country_name AS country_name,
        country_code AS country_code,
        utc_original AS utc_original,
        utc_hora AS utc,
        utc_segundos AS utc_segundos
FROM {tabla_datos_output_view}
WHERE country_code IN ('BO','EC','CL','VE','AR','CR','DO','SV','GT','HN',
'NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR') 
''')
cursor.execute(query).as_pandas()
print(f" Generación de: {tabla_datos_output_table} ")

 Generación de: utc_latam 


In [9]:
query = f'''SELECT country_code,
        COUNT(*) as n_obs
FROM {tabla_datos_output_table}
GROUP BY country_code 
ORDER BY country_code '''
cursor.execute(query).as_pandas()

,country_code,n_obs
0,AR,1
1,BO,1
2,BR,1
3,BZ,1
4,CL,1
5,CO,1
6,CR,1
7,DO,1
8,EC,1
9,GT,1
